In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
## modeling
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [3]:
df=pd.read_csv('data\stud.csv')
df.head()


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
## X and y variable
X=df.drop(columns=['math_score'],axis=1)

In [5]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
y=df['math_score']

In [7]:
## create columns transformer with 3 types of transformer

num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler

from sklearn.compose import ColumnTransformer
numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("oneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features),
    ]
)

In [8]:
X=preprocessor.fit_transform(X)

In [9]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [12]:
X_train.shape

(800, 19)

In [17]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [21]:
models={
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-NeighborsRegressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "CatBoosting Regressor":CatBoostRegressor(verbose=False),
    "AdaBoost Regressor":AdaBoostRegressor()
}
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train) ## train model

    ## make prediction
    y_train_pred=model.predict(X_train)

    y_test_pred=model.predict(X_test)

    ##  evaluate train and test dataset
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)
    

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('model  performances for training set')
    print('Root mean squared error: {:.4f}'.format(model_train_rmse))

    print('mean absolute error :{:.4f}'.format(model_train_mae))
    print('r2 score: {:.4f}'.format(model_train_r2))

    print('............')
    print('model performance test')

    print('Root mean squared error: {:.4f}'.format(model_test_rmse))
    print('mean absolute error: {:.4f}'.format(model_test_mae))
    print('r2 score: {:.4f}'.format(model_test_r2))

    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')





Linear Regression
model  performances for training set
Root mean squared error: 5.3293
mean absolute error :4.2715
r2 score: 0.8740
............
model performance test
Root mean squared error: 5.4252
mean absolute error: 4.2222
r2 score: 0.8790


Lasso
model  performances for training set
Root mean squared error: 6.5938
mean absolute error :5.2063
r2 score: 0.8071
............
model performance test
Root mean squared error: 6.5197
mean absolute error: 5.1579
r2 score: 0.8253


Ridge
model  performances for training set
Root mean squared error: 5.3233
mean absolute error :4.2650
r2 score: 0.8743
............
model performance test
Root mean squared error: 5.3904
mean absolute error: 4.2111
r2 score: 0.8806


K-NeighborsRegressor
model  performances for training set
Root mean squared error: 5.7079
mean absolute error :4.5168
r2 score: 0.8555
............
model performance test
Root mean squared error: 7.2530
mean absolute error: 5.6210
r2 score: 0.7838


Decision Tree
model  performances

In [22]:
## results
pd.DataFrame(list(zip(model_list,r2_list)),columns=['model name','r2 score']).sort_values(by=['r2 score'],ascending=False)

,model name,r2 score
2,Ridge,0.880593
0,Linear Regression,0.879046
7,CatBoosting Regressor,0.851632
5,Random Forest Regressor,0.850560
8,AdaBoost Regressor,0.848654
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-NeighborsRegressor,0.783813
4,Decision Tree,0.776135


In [23]:
## linear Regression
lin_model=LinearRegression(fit_intercept=True)
lin_model=lin_model.fit(X_train,y_train)
y_pred=lin_model.predict(X_test)
score=r2_score(y_test,y_pred)*100
print('accuracy of model  is %.2f' %score)

accuracy of model  is 87.90


In [24]:
pred_df=pd.DataFrame({'actual value':y_test,'predicted value':y_pred,'Difference':y_test-y_pred})
pred_df

,actual value,predicted value,Difference
521,91,76.4375,14.5625
737,53,59.3125,-6.3125
740,80,76.6250,3.3750
660,74,76.5625,-2.5625
411,84,87.7500,-3.7500
...,...,...,...
408,52,43.6875,8.3125
332,62,62.3125,-0.3125
208,74,67.7500,6.2500
613,65,67.0000,-2.0000
